In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_generation_I_Pok%C3%A9mon"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [3]:
tables = soup.find_all("table")

# Search for the table with the specific caption
target_table = None
for table in tables:
    if table.caption and "List of Pokémon species introduced in Generation I (1996)" in table.caption.get_text():
        target_table = table
        break

In [4]:
df = pd.read_html(str(target_table))[0]
df.head()

Name                      National Pokédex number Type(s)             
      English             Japanese National Pokédex number Primary Secondary   
0   Bulbasaur  Fushigidane (フシギダネ)                    0001   Grass    Poison  \
1     Ivysaur    Fushigisō (フシギソウ)                    0002   Grass    Poison   
2    Venusaur  Fushigibana (フシギバナ)                    0003   Grass    Poison   
3  Charmander      Hitokage (ヒトカゲ)                    0004    Fire      Fire   
4  Charmeleon        Rizādo (リザード)                    0005    Fire      Fire   

             Evolves from               Evolves into   
             Evolves from               Evolves into   
0  Beginning of evolution             Ivysaur (#002)  \
1        Bulbasaur (#001)            Venusaur (#003)   
2          Ivysaur (#002)  Mega Evolution Gigantamax   
3  Beginning of evolution          Charmeleon (#005)   
4       Charmander (#004)           Charizard (#006)   

                                               Notes  
                                               Notes  
0  It uses the nutrients that are stored in the s...  
1  When it's exposed to sunlight, the seed, now a...  
2  Its English name is a portmanteau of "Venus" (...  
3  Charmander is a bipedal, salamander-like creat...  
4  Charmeleon is similar to Charmander yet larger...

In [5]:
df.columns

MultiIndex([(                   'Name',                 'English'),
            (                   'Name',                'Japanese'),
            ('National Pokédex number', 'National Pokédex number'),
            (                'Type(s)',                 'Primary'),
            (                'Type(s)',               'Secondary'),
            (           'Evolves from',            'Evolves from'),
            (           'Evolves into',            'Evolves into'),
            (                  'Notes',                   'Notes')],
           )

In [18]:
# assume your DataFrame is named `df`
cols_to_keep = [('Name', 'English'),('National Pokédex number','National Pokédex number'), ('Type(s)', 'Primary'), ('Type(s)', 'Secondary'), ('Evolves from', 'Evolves from'), ('Evolves into', 'Evolves into'), ('Notes', 'Notes')]

# select only the relevant columns using loc[]
df_subset = df.loc[:, cols_to_keep]

# flatten the MultiIndex by joining the column levels with '_'
df_subset.columns = ['_'.join(col).strip() for col in df_subset.columns.values]

# rename the columns as desired
df_subset = df_subset.rename(columns={'Name_English': 'Name',
                                      'National Pokédex number_National Pokédex number':'PokeIndex',
                                      'Type(s)_Primary': 'Primary type',
                                      'Type(s)_Secondary': 'Secondary type',
                                      'Evolves from_Evolves from': 'Evolves from',
                                      'Evolves into_Evolves into': 'Evolves into',
                                      'Notes_Notes': 'Notes'})

In [19]:
df_subset.head()

,Name,PokeIndex,Primary type,Secondary type,Evolves from,Evolves into,Notes
0,Bulbasaur,0001,Grass,Poison,Beginning of evolution,Ivysaur (#002),It uses the nutrients that are stored in the s...
1,Ivysaur,0002,Grass,Poison,Bulbasaur (#001),Venusaur (#003),"When it's exposed to sunlight, the seed, now a..."
2,Venusaur,0003,Grass,Poison,Ivysaur (#002),Mega Evolution Gigantamax,"Its English name is a portmanteau of ""Venus"" (..."
3,Charmander,0004,Fire,Fire,Beginning of evolution,Charmeleon (#005),"Charmander is a bipedal, salamander-like creat..."
4,Charmeleon,0005,Fire,Fire,Charmander (#004),Charizard (#006),Charmeleon is similar to Charmander yet larger...


In [21]:
Pokemons=[]

for row in df_subset.iterrows():
    data=row[1]
    dataDict={
            "PokeIndex":data['PokeIndex'],
            "name":data['Name'],
            "Primary type":data['Primary type'],
            "Secondary type":data['Secondary type'],
            "Evolves from":data['Evolves from'],
            "Evolves into":data['Evolves into'],
            "Notes":data['Notes']
    }
    Pokemons.append(dataDict)

In [22]:
len(Pokemons)

152

In [23]:
import json

with open('PokemonDataSet.json','w') as f:
    json.dump(Pokemons,f)